In [1]:
import yadisk
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import os
import time
import sys
import math
from collections import Counter
import posixpath
import shutil
from os import listdir
from os.path import isfile, join
import asyncio

In [2]:
input_path = "E:/output/"
input_dir = "E:/input/"
download_path = "/архив/readable_bg/"
upload_path = "/архив/_result/readable_bg/batches/"

In [2]:
async def getBatchAsync(batch_id: int, client: yadisk.AsyncClient(),*, batch_size=100, n_parallel_requests=10):
  async def download_files(queue):
    while queue:
      from_path, to_path = queue.pop(0)

      #print(f"Uploading {in_path} -> {out_path}")

      try:
        await client.download(from_path, to_path, timeout=180.0)
      except yadisk.exceptions.PathExistsError:
        print(f"{to_path} already exists")

  start_el = batch_size * batch_id
  from_dir = f"{input_path}batch_{batch_id}/"
  if os.path.exists(from_dir) is False:
    os.mkdir(from_dir, mode = 0o777)
  if os.path.exists(input_dir) is False:
    os.mkdir(input_dir, mode = 0o777)
  if os.path.exists(from_dir + "checking/") is False:
    os.mkdir(from_dir + "checking/", mode = 0o777)
  if os.path.exists(from_dir + "res/") is False:
    os.mkdir(from_dir + "res/", mode = 0o777)
  download_queue = []
  for i in range(batch_size):
    if i+start_el >= len(readable_bg_names):
      break
    if os.path.exists(f"{input_dir}{readable_bg_names[i+start_el]}") is False:
      download_queue.append((f"{download_path}{readable_bg_names[i+start_el]}", f"{input_dir}{readable_bg_names[i+start_el]}"))
  tasks = [download_files(download_queue) for i in range(n_parallel_requests)]
  await asyncio.gather(*tasks)
  return from_dir

In [10]:
async def recursive_upload(from_dir: str, to_dir: str, client: yadisk.AsyncClient(), n_parallel_requests: int = 5):
    async def upload_files(queue):
        while queue:
            in_path, out_path = queue.pop(0)

            #print(f"Uploading {in_path} -> {out_path}")

            try:
                await client.upload(in_path, out_path, timeout=120.0)
            except yadisk.exceptions.PathExistsError:
                print(f"{out_path} already exists")

    async def create_dirs(queue):
        while queue:
            path = queue.pop(0)

            #print(f"Creating directory {path}")

            try:
                await client.mkdir(path, timeout=120.0)
            except yadisk.exceptions.PathExistsError:
                print(f"{path} already exists")

    mkdir_queue = []
    upload_queue = []

    #print(f"Creating directory {to_dir}")

    try:
        await client.mkdir(to_dir, timeout=120.0)
    except yadisk.exceptions.PathExistsError:
        print(f"{to_dir} already exists")

    for root, dirs, files in os.walk(from_dir):
        rel_dir_path = root.split(from_dir)[1].strip(os.path.sep)
        rel_dir_path = rel_dir_path.replace(os.path.sep, "/")
        dir_path = posixpath.join(to_dir, rel_dir_path)

        for dirname in dirs:
            mkdir_queue.append(posixpath.join(dir_path, dirname))

        for filename in files:
            out_path = posixpath.join(dir_path, filename)
            rel_dir_path_sys = rel_dir_path.replace("/", os.path.sep)
            in_path = os.path.join(from_dir, rel_dir_path_sys, filename)

            upload_queue.append((in_path, out_path))

        tasks = [upload_files(upload_queue) for i in range(n_parallel_requests)]
        tasks.extend(create_dirs(mkdir_queue) for i in range(n_parallel_requests))

        await asyncio.gather(*tasks)
    #shutil.rmtree(from_dir)

In [5]:
def sortMasksByPerimeter(masks):
    if len(masks) == 0:
        return masks
    sorted_masks = sorted(masks, key=(lambda x: (x['bbox'][2] - x['bbox'][0]) * (x['bbox'][3] - x['bbox'][1])), reverse=True)
    return sorted_masks

In [6]:
def sortMasksByArea(masks):
    if len(masks) == 0:
        return masks
    sorted_masks = sorted(masks, key=(lambda x: x['area']), reverse=True)
    return sorted_masks

In [7]:
def checkResolution(image, maxres):
    h = image.shape[0]
    w = image.shape[1]
    if h > maxres and h > w:
        hn = math.ceil(h*maxres/h)
        wn = math.ceil(w*maxres/h)
        return cv2.resize(image, (wn, hn), cv2.INTER_AREA)
    elif w > maxres:
        hn = math.ceil(h*maxres/w)
        wn = math.ceil(w*maxres/w)
        return cv2.resize(image, (wn, hn), cv2.INTER_AREA)
    return image

In [8]:
def SaveAllMasks(image, masks, path, ext):
    if len(masks) == 0:
        return
    sorted_masks = sortMasksByArea(masks)[:8]
    #sorted_masks = sortMasksByArea(masks)
    h = image.shape[0]
    w = image.shape[1]
    c = 4
    i = 1
    color_not_mask = (255, 255, 255, 0)
    color_mask = (255, 255, 255, 255)
    for ann in sorted_masks:
        img = np.zeros((h, w, c), dtype=np.uint8)
        m = ann['segmentation']
        img[m] = color_mask
        not_m = np.invert(m)
        img[not_m] = color_not_mask
        img = cv2.GaussianBlur(img, (11, 11), 0)
        cv2.imwrite(path + '/' + f'_({i})' + ext, img)
        i += 1

In [9]:
def check_mask(mask, biggest_contour):
    h = mask.shape[0]
    w = mask.shape[1]
    sh = sorted(list(biggest_contour), key=(lambda x: x[0][1]), reverse=True)
    if (sh[0][0][1] - sh[len(sh)-1][0][1]) >= h - 5:
        return True
    sw = sorted(list(biggest_contour), key=(lambda x: x[0][0]), reverse=True)
    if (sw[0][0][0] - sw[len(sw)-1][0][0]) >= w - 5:
        return True
    return False

In [10]:
def get_biggest_contour(contours):
    biggest_area = 0
    biggest_contour = None
    for i in range(len(contours)):
        cnt = contours[i]
        area = cv2.contourArea(cnt)
        if area > biggest_area:
            biggest_area = area
            biggest_contour = cnt
    return biggest_contour

In [11]:
def checking_lines(mask, thickness=7, available=0.9):
    h = mask.shape[0]
    w = mask.shape[1]
    checker = False
    counter_h = 0
    counter_l = 0
    lines_mask = np.zeros(mask.shape, np.bool_)
    for x in range(w):
        for y in range(thickness):
            if mask[y][x] == False:
                counter_h += 1
                lines_mask[y][x] = True
            if mask[h-y-1][x] == False:
                counter_l += 1
                lines_mask[h-y-1][x] = True
    #print(counter_h, counter_l, (available * thickness * w))
    if counter_h >= (available * thickness * w):
            checker = True
    if counter_l >= (available * thickness * w) and checker is True:
            return (checker,lines_mask)
    
    checker = False
    counter_r = 0
    counter_l = 0
    lines_mask = np.zeros(mask.shape, np.bool_)
    for y in range(h):
        for x in range(thickness):
            if mask[y][x] == False:
                counter_r += 1
                lines_mask[y][x] = True
            if mask[y][w-x-1] == False:
                counter_l += 1
                lines_mask[y][w-x-1] = True
    #print(counter_r, counter_l, (available * thickness * h))
    if counter_r >= (available * thickness * h):
            checker = True
    if counter_l < (available * thickness * h):
            checker = False
    return (checker,lines_mask)

In [25]:
def selectAndDeleteMask(image, masks, path, ext, name):
    if len(masks) == 0:
        return None
    
    color_not_mask = (255, 255, 255, 0)
    color_mask = (255, 255, 255, 255)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2RGBA)
    mask_image = np.zeros((image.shape[0], image.shape[1], image.shape[2]), dtype=np.uint8)

    # checking bigest mask
    h = image.shape[0]
    w = image.shape[1]
    available = 0.9
    main_mask_box = masks[0]['bbox']
    checker = False
    if ((main_mask_box[3] - main_mask_box[1]) >= h * available) and ((main_mask_box[2] - main_mask_box[0]) >= w * available):
        checker = True

    if checker == False:
        print("scenerio")
        main_mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.bool_)
        for i in range(0, len(masks)):
            main_mask[masks[i]['segmentation']] = True
        mask_image[main_mask] = color_not_mask
        main_mask = np.invert(main_mask)
        mask_image[main_mask] = color_mask
    else:
        main_mask = masks[0]['segmentation']
        not_m = np.invert(main_mask)
        mask_image[main_mask] = color_mask
        mask_image[not_m] = color_not_mask
    
    # check lines
    mask_image = cv2.GaussianBlur(mask_image, (11, 11), 0)
    main_mask = np.all(mask_image[...,:4] == color_mask, axis=-1)

    thickness = 5
    available = 0.5
    check, lines_mask = checking_lines(main_mask, thickness=thickness, available=available)
    while check == True and thickness <= 12:
        thickness += 1
        tmp_check, tmp_lines_mask = checking_lines(main_mask, thickness=thickness, available=available)
        if tmp_check == False:
            thickness -= 1
            break
        else:
            check = tmp_check
            lines_mask = tmp_lines_mask
    #print("thickness: ", thickness)
    if check == True:
        mask_image[lines_mask] = color_mask
        main_mask[lines_mask] = True
    
    cv2.imwrite(path + '/' + name + '_mask' + ext, mask_image)
    image[main_mask, 3] = 0
    return (image, mask_image)

In [13]:
def make_all_calcs_and_save(path_to_read, dir_to_save, name, id, ext, maxres, mask_generator):
    if os.path.exists(path_to_read) is False:
        print(path_to_read, " --- doesn't exist")
        return
    path_cur = f"{dir_to_save}{name}({id})" 
    #print('iter:', id, '   file:', name + ext)
    image = cv2.imread(path_to_read)
    try:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    except cv2.error:
        print('problem with image: ', cv2.error.msg)
        return
    image = checkResolution(image, maxres)
    img_start = cv2.cvtColor(image, cv2.COLOR_RGB2RGBA)
    masks = mask_generator.generate(image)
    masks = sortMasksByPerimeter(masks)
    if os.path.exists(path_cur) is False:
        os.mkdir(path_cur, mode = 0o777)

    image, mask = selectAndDeleteMask(image, masks, path_cur, ext, f"{name}({id})")
    SaveAllMasks(image, masks, path_cur, ext)
    collage = np.hstack((image, mask))
    collage = np.hstack((collage, img_start))
    collage = cv2.cvtColor(collage, cv2.COLOR_BGRA2RGBA)
    cv2.imwrite(f"{dir_to_save}checking/{name}({id}){ext}", collage)

    image = cv2.cvtColor(image, cv2.COLOR_BGRA2RGBA)
    cv2.imwrite(f"{dir_to_save}res/{name}({id}){ext}", image)

In [14]:
print("Torch version:",torch.__version__)

Torch version: 2.2.2+cu121


In [15]:
print("Is CUDA enabled?",torch.cuda.is_available())

Is CUDA enabled? True


In [16]:
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

#sam_checkpoint = "drive/MyDrive/SAM/sam_vit_l_0b3195.pth"
sam_checkpoint = "sam_vit_l_0b3195.pth"
#sam_checkpoint = "drive/MyDrive/SAM/sam_vit_h_4b8939.pth"
model_type = "vit_l"
#model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

1) список всех изображений, которые с ошибкой
   элементы списка имеют формат: (путь, имя элемента вместе с порядковым номером)
- так же отдельный список элементов, для которых по каким-то причинам созданы маски, но не созданы файлы в res и в checking
- так же отдельный список элементов, для которых по какой-то причине была создана отдельная папка для масок, но не было создано ни масок, ни, соответственно, файлов в res и в checking

In [40]:
start_dir = 'E:/output/'
start_dir_names = os.listdir(start_dir)

In [41]:
start_dir_numbers = list(map(lambda x: int(x[x.rfind("_")+1:]), start_dir_names))
m = max(start_dir_numbers)

In [42]:
list_of_inv_bg = [
    # path to file, name
]
list_of_missed_res_check = [
    # path to file, name
]
list_of_missed_masks = [
    # path to file, name
] # hope it is empty

In [43]:
t = time.time()

for i in range(m):
    cur_path = start_dir+start_dir_names[i]
    batch_names = os.listdir(cur_path+'/')
    print(cur_path, ' || ', time.time() - t, " sec")
    indexes = []
    for i in range(len(batch_names)):
        if batch_names[i] == 'checking' or batch_names[i] == 'res':
            indexes.append(i)
    ind = 0
    for el in indexes:
        batch_names.pop(el + ind)
        ind -= 1
    
    for el in batch_names:
        el_path = f"{cur_path}/{el}/{el}_mask.png"
        if os.path.exists(el_path) is True:
            #print(el)
            mask_image = cv2.imread(el_path)
            mask_image_gray = cv2.cvtColor(mask_image, cv2.COLOR_BGRA2GRAY)
            contours, hierarchy = cv2.findContours(mask_image_gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            biggest_contour = get_biggest_contour(contours)
            if check_mask(mask_image, biggest_contour) is False:
                #print(el, '    ', mask_image.shape)
                list_of_inv_bg.append((el_path, el))
            else:
                if os.path.exists(f"{cur_path}/res/{el}.png") == False \
                    and os.path.exists(f"{cur_path}/res/{el}_corr.png") == False\
                    or os.path.exists(f"{cur_path}/checking/{el}.png") == False \
                    and os.path.exists(f"{cur_path}/checking/{el}_corr.png") == False:
                    list_of_missed_res_check.append((el_path, el))
        else:
            list_of_missed_masks.append((el_path, el))
print(time.time() - t, " sec")

E:/output/batch_0  ||  0.0  sec
E:/output/batch_1  ||  2.2875144481658936  sec
E:/output/batch_10  ||  4.343485355377197  sec
E:/output/batch_100  ||  5.803813219070435  sec
E:/output/batch_101  ||  7.94929575920105  sec
E:/output/batch_102  ||  9.951745748519897  sec
E:/output/batch_103  ||  12.01330304145813  sec
E:/output/batch_104  ||  14.008751153945923  sec
E:/output/batch_105  ||  16.121811866760254  sec
E:/output/batch_106  ||  18.28680419921875  sec
E:/output/batch_107  ||  20.271366119384766  sec
E:/output/batch_108  ||  22.165791988372803  sec
E:/output/batch_109  ||  24.081237316131592  sec
E:/output/batch_11  ||  25.837644577026367  sec
E:/output/batch_110  ||  27.417999744415283  sec
E:/output/batch_111  ||  29.202460050582886  sec
E:/output/batch_112  ||  30.849831104278564  sec
E:/output/batch_113  ||  32.483211040496826  sec
E:/output/batch_114  ||  34.27861523628235  sec
E:/output/batch_115  ||  35.98855996131897  sec
E:/output/batch_116  ||  37.59992241859436  sec
E:

In [44]:
print(len(list_of_inv_bg))

1432


In [45]:
print(len(list_of_missed_res_check))

0


In [46]:
for el in list_of_missed_res_check:
    print(el)

In [47]:
print(len(list_of_missed_masks))

0


In [48]:
print(list_of_missed_masks)

[]


2) список всех изображений, которые нужно обработать

In [17]:
input_images = []
input_dir = "E:/input/"

In [18]:
input_images = os.listdir(input_dir)
len(input_images)

8607

3) список всех изображений, которые уже обработаны

In [51]:
worked_images = []
readable_path = "/архив/readable_bg/"

In [53]:
with yadisk.Client("---", "---", "---") as client:
    worked_images = list(client.listdir(readable_path, timeout=120.0))

7639

In [55]:
worked_images_names = list(map(lambda x: x['name'], worked_images))

In [56]:
print(len(worked_images_names))

7639


4.запуск вычислений

просто запустить вычисления в том виде, в каком оно есть, с теми найденными файлами, которые есть
сохранить все файлы res и checking из всех батчей в какие-то папки, где они будут лежать вместе
уже после всего этого проверять, всё ли обработалось, всё ли корректно (там должно будет быть проще)

In [19]:
ext = '.png'
maxres = 1024
to_dir = "E:/output_new/"
input_path = "E:/input/"
batch_id = 152
batch_size = 50

In [26]:
i = 0
id = 0
i_max = 10
start_i = 22
t = time.time()
count_of_inv = 0
count_of_correct = 0
for el in input_images:
    if i < start_i:
        i += 1
        id += 1
        continue
    path = f"{input_path}{el}"
    el_name = el[:el.rfind('.')]
    # checker = False
    # for worked_el in worked_images_names:
    #     worked_name = worked_el[:worked_el.rfind('(')]
    #     if el_name == worked_name:
    #         checker = True
    #         print("in worked_images_names,", checker)
    #         break
    # if checker == True:
    #     checker = False
    #     w_el = ""
    #     w_el_path = ""
    #     for worked_el in list_of_inv_bg:
    #         worked_name = worked_el[1][:worked_el.rfind('(')]
    #         if el_name == worked_name:
    #             checker = True
    #             w_el = worked_el[1]
    #             w_el_path = worked_el[0]
    #             print("in list_of_inv_bg,", checker)
    #             break
    #     if checker == True:
    #         dir_to_save = w_el_path[:w_el_path.rfiind('/')]
    #         dir_to_save = dir_to_save[:dir_to_save.rfiind('/')] + '/'
    #         w_id = int(w_el[w_el.rfind('(')::w_el.rfind(')')])
    #         print(f"{path} with id: {w_id} has inv BD, i: {i}, dir_to_save: {dir_to_save}, w_id: {w_id}")
    #         count_of_inv += 1
    #         #make_all_calcs_and_save(path, dir_to_save, el_name, w_id, ext, maxres, mask_generator)
    #         i += 1
    #         print(time.time()-t)
    #         continue
    #     else:
    #         print('iter:', id, ' file:', name + name_ext, '   continue, it already exists, i:', i)
    #         print("not in list_of_inv_bg,", checker)
    #         i += 1
    #         count_of_correct += 1
    #         print(time.time()-t)
    #         continue
    if os.path.exists(path) is False:
        print(path, " --- file doesn't exist, i:", i)
        i += 1
        print(time.time()-t)
        continue
    name_with_id = f"{el_name}({id})"
    path_cur = to_dir + name_with_id
    if os.path.exists(path_cur) is True:
        print(path_cur, '   continue, file has worked already, i:', i)
        id+=1
        i += 1
        print(time.time()-t)
        continue
    make_all_calcs_and_save(path, to_dir, el_name, id, ext, maxres, mask_generator)
    print('iter:', id, ' file:', el_name + ext, '    worked, it wasn\'t exist', i)
    id += 1
    i += 1
    # if i >= i_max + start_i:
    #     break
    print(time.time()-t)
print('i:', i)
print('count_of_inv', count_of_inv)
print('count_of_correct', count_of_correct)

iter: 22  file: 61 105 25.png     worked, it wasn't exist 22
6.048359632492065
iter: 23  file: 61 105 26.png     worked, it wasn't exist 23
14.430247783660889
iter: 24  file: 61 105 27.png     worked, it wasn't exist 24
21.46782636642456
iter: 25  file: 61 105 27_02.png     worked, it wasn't exist 25
27.40015959739685
iter: 26  file: 61 105 28.png     worked, it wasn't exist 26
35.198912620544434
iter: 27  file: 61 105 29.png     worked, it wasn't exist 27
41.45431923866272
iter: 28  file: 61 105 29_02.png     worked, it wasn't exist 28
47.39716362953186
iter: 29  file: 61 105 4.png     worked, it wasn't exist 29
55.45897603034973
iter: 30  file: 61 105 6.png     worked, it wasn't exist 30
66.03035235404968
iter: 31  file: 61 105 6_02.png     worked, it wasn't exist 31
75.23042130470276
iter: 32  file: 61 105 8.png     worked, it wasn't exist 32
81.10574197769165
iter: 33  file: 61 105 9.png     worked, it wasn't exist 33
88.74846005439758
iter: 34  file: 61 105 9_02.png     worked, it

выгрузка на яндекс диск

In [12]:
from_dir = "E:/output_new/"
to_dir = "/архив/result/"
token = "y0_AgAAAAAwXi3yAAuiHAAAAAECOWVuAAAtLJ9XZQlPN7OltYYM0aLrgoxKmA"
n_parallel_requests = 20

In [5]:
client = yadisk.AsyncClient(token=token)

In [11]:
await recursive_upload(from_dir, to_dir, client, n_parallel_requests = n_parallel_requests)

/архив/result/ already exists
/архив/result/61 105 11_02(2) already exists
/архив/result/61 105 18(7) already exists
/архив/result/61 105 16_02(5) already exists
/архив/result/61 105 14(3) already exists
/архив/result/61 105 17(6) already exists
/архив/result/61 105 1(0) already exists
/архив/result/61 105 11(1) already exists
/архив/result/61 105 18_03(9) already exists
/архив/result/61 105 16(4) already exists
/архив/result/61 105 18_02(8) already exists
/архив/result/61 105 19_02(14) already exists
/архив/result/61 105 18_04(10) already exists
/архив/result/61 105 19(13) already exists
/архив/result/61 105 18_06(12) already exists
/архив/result/61 105 1_02(15) already exists
/архив/result/61 105 20(16) already exists
/архив/result/61 105 18_05(11) already exists
/архив/result/61 105 20_02(17) already exists
/архив/result/61 105 23(20) already exists
/архив/result/61 105 24(21) already exists
/архив/result/61 105 21_02(19) already exists
/архив/result/61 105 27(24) already exists
/ар

CancelledError: 

In [13]:
await client.close()

In [17]:
ext = '.png'
maxres = 1024
to_dir = "E:/corrected/"
token = 'y0_AgAAAAAwXi3yAAuiHAAAAAECOWVuAAAtLJ9XZQlPN7OltYYM0aLrgoxKmA'
n_parallel_requests_download = 5
n_parallel_requests_upload = 5
client = yadisk.AsyncClient(token=token)
input_dir = "E:/input/"
download_path = "/архив/readable_bg/"

In [18]:
images = [
    # (path, id)
    # old
    #(f"{input_dir}61 65 424.jpg", 5280),
    #(f"{input_dir}61 65 464_03.jpg", 5333),
    #(f"{input_dir}61 66 137.jpg", 5639),
    #(f"{input_dir}61 42 990.jpg", 750),
    #(f"{input_dir}61 61 315.jpg", 3936)
    # new
    (f"{input_dir}61 42 444.jpg", 603),
    (f"{input_dir}61 42 543.jpg", 631),
    (f"{input_dir}61 42 557.jpg", 637)
]

In [23]:
t = time.time()
#client = yadisk.AsyncClient(token=token)

for el in images:
    path = el[0]
    id = el[1]
    if os.path.exists(path) is False:
        print(path, " --- doesn't exist")
        continue
    name = path[path.rfind('/'):]
    name_ext = name[name.rfind('.'):]
    #name = name[:name.rfind('.')] + '(' + str(id) + ')'
    name = name[:name.rfind('.')]
    path_cur = to_dir + name + '(' + str(id) + ')'
    if os.path.exists(path_cur) is True:
        print(path_cur)
        print('continue')
        id+=1
        continue
    #print('iter:', id, ' file:', name + name_ext)
    make_all_calcs_and_save(path, to_dir, name, id, ext, maxres, mask_generator)
    # #image = cv2.imread(f"{input_dir}{images_paths[i]}")
    # image = cv2.imread(path)
    # try:
    #     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # except cv2.error:
    #     print('problem with image: ', cv2.error.msg)
    #     id+=1
    #     continue
    # image = checkResolution(image, maxres)
    # img_start = cv2.cvtColor(image, cv2.COLOR_RGB2RGBA)
    # #img_start = cv2.cvtColor(img_start, cv2.COLOR_RGBA2BGRA)
    # masks = mask_generator.generate(image)
    # masks = sortMasksByPerimeter(masks)
    # if os.path.exists(path_cur) is False:
    #     os.mkdir(path_cur, mode = 0o777)

    # image, mask = deleteLargestBoxMask(image, masks, path_cur, ext, name)
    # collage = np.hstack((image, mask))
    # collage = np.hstack((collage, img_start))
    # collage = cv2.cvtColor(collage, cv2.COLOR_BGRA2RGBA)
    # cv2.imwrite(to_dir + "checking/" + name + ext, collage)

    # image = cv2.cvtColor(image, cv2.COLOR_BGRA2RGBA)
    # SaveAllMasks(image, masks, path_cur, ext)
    # cv2.imwrite(to_dir + '/res/' + name + ext, image)
    print(time.time()-t)

iter: 603    file: /61 42 444.png
0 0 2287.5
41 0 2287.5
9.823059320449829
E:/corrected//61 42 543(631)
continue
E:/corrected//61 42 557(637)
continue


In [ ]:
t = time.time()
client = yadisk.AsyncClient(token=token)

for el in images:
    

#for i in range(id, len(images_paths)):
for i in range(id, id+batch_size):
    if id % batch_size == 0:
        print("new batch_id:", batch_id)
        if os.path.exists(f"{input_path}batch_{batch_id}/") is True:
            to_dir = f"{upload_path}batch_{batch_id}/"
            print("Uploading to:")
            print(to_dir)
            #sendBatch(from_dir, to_dir, batch_id, batch_size)
            #recursive_upload(from_dir: str, to_dir: str, client, n_parallel_requests: int = 5)
            #await recursive_upload(from_dir, to_dir, client, n_parallel_requests=n_parallel_requests_upload)
            batch_id += 1
        #getBatchAsync(batch_id: int, token: str, batch_size=100, n_parallel_requests=10)
        #from_dir = getBatch(batch_id, batch_size)
        from_dir = await getBatchAsync(batch_id, client, batch_size=batch_size, n_parallel_requests=n_parallel_requests_download)
    path = f"{from_dir}{images_paths[i]}"
    name = images_paths[i]
    name_ext = name[name.rfind('.'):]
    name = name[:name.rfind('.')] + '(' + str(id) + ')'
    path_cur = from_dir + name
    if os.path.exists(path_cur) is True:
        print(path_cur)
        print('continue')
        id+=1
        continue
    print('iter:', id, ' batch:', batch_id, ' file:', name + name_ext)
    #print(time.time()-t)

    image = cv2.imread(f"{input_dir}{images_paths[i]}")
    try:
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    except cv2.error:
      print('problem with image: ', cv2.error.msg)
      id+=1
      continue
    image = checkResolution(image, maxres)
    img_start = cv2.cvtColor(image, cv2.COLOR_RGB2RGBA)
    #img_start = cv2.cvtColor(img_start, cv2.COLOR_RGBA2BGRA)
    masks = mask_generator.generate(image)
    masks = sortMasksByPerimeter(masks)
    if os.path.exists(path_cur) is False:
      os.mkdir(path_cur, mode = 0o777)
    #if os.path.exists(path_cur[:path_cur.rfind('.')]) is False:
        #os.mkdir(path_cur[:path_cur.rfind('.')], mode = 0o777)
        #print(path_cur[:path_cur.rfind('.')])
    #print(time.time()-t)

    image, mask = deleteLargestBoxMask(image, masks, path_cur, ext, name)
    collage = np.hstack((image, mask))
    collage = np.hstack((collage, img_start))
    collage = cv2.cvtColor(collage, cv2.COLOR_BGRA2RGBA)
    cv2.imwrite(from_dir + "checking/" + name + ext, collage)

    image = cv2.cvtColor(image, cv2.COLOR_BGRA2RGBA)
    SaveAllMasks(image, masks, path_cur, ext)
    cv2.imwrite(from_dir + '/res/' + name + ext, image)
    id+=1
    print(time.time()-t)